In [4]:
from selenium import webdriver
import re
import pandas as pd
import os
import time
global chrome_driver_dir

if not os.path.isdir('./Web_Article/'):
    os.mkdir('./Web_Article/')

chrome_driver_dir='C:/Chromedriver/chromedriver.exe'
end_time='2020.12'

inews_regex='\\[아이뉴스.*?\\]'

do_all=False

if not os.path.isdir('./Web_Article/inews24/'):
    os.mkdir('./Web_Article/inews24/')
try:
    all_data=pd.read_excel('./Web_Article/inews24/inews24_기자수첩.xlsx',keep_default_na=False)
    all_title=all_data["제목"]
    all_url=all_data["URL"]
except:
    all_data=pd.DataFrame()
    all_title = []
    all_url = []
driver=webdriver.Chrome(executable_path=chrome_driver_dir)
# main_url='https://www.inews24.com/list/020100?page='
# main_url='https://www.inews24.com/list/opinion?page='
main_url='https://www.inews24.com/list/reporter?page='
i = 1
#수집할 기사의 '목록'을 선별함
## 목록을 선별한뒤에 각 기사에 접근하여 크롤링으함
target_article_df=pd.DataFrame()
#False일 경우 다음페이지 진행 계속
stop_page=False
before_title_list=[]
while True:
    target_page_url=main_url+str(i)
    print("inews24 - it 오피니언", i, "번째 페이지 확인중 ...")
    driver.get(target_page_url)
    #목록을 조회할때는 제목 / 날짜 / url만 체크

    #다음페이지가 없어서 강제로 다른페이지로 이동하면 초기화
    time.sleep(1)
    # 1페이지만 Header 뉴스가 있으므로 따로처리함

    title_in_list=[]
    url_in_list=[]
    date_in_list=[]


    if i==1:
        news_header_info = driver.find_element_by_css_selector('article > header')
        header_title = news_header_info.find_element_by_css_selector('div > a > h1').text
        header_url = news_header_info.find_element_by_css_selector('div > a').get_attribute('href')
        header_date = news_header_info.find_element_by_css_selector('div > time').text
        md_header_date = header_date[:10]
        title_in_list.append(header_title)
        url_in_list.append(header_url)
        date_in_list.append(md_header_date)

    #
    #방법 1. 기사 하나의 정보를 가지고 온 뒤에 그 안에서 제목,날짜, url을 체크하는 방법
    news_info_in_list=driver.find_elements_by_css_selector('ol > li.list > div')

    if len(news_info_in_list)==0:
        break
    for j in range(len(news_info_in_list)):

        this_news_info=news_info_in_list[j]

        #정보안에서 제목 찾기
        this_titl_html=this_news_info.find_element_by_css_selector('a')
        this_title=this_titl_html.text
        title_in_list.append(this_title)

        #find_element_by_css_selector는 한개의 값을 변수로 저장하고
        #find_elements_by_css_selector는 한개 이상의 값을 리스트 형태로 저장한다.

        #정보 안에서 url 찾기
        this_url_html=this_news_info.find_element_by_css_selector('a')
        this_url=this_url_html.get_attribute('href')
        url_in_list.append(this_url)

        #정보 안에서 날짜 찾기

        this_date_in_html=this_news_info.find_element_by_css_selector('time')
        this_date=this_date_in_html.text
        this_date_md = this_date[:10]
        date_in_list.append(this_date_md)



    this_page_article_info=pd.DataFrame({"날짜":date_in_list, "제목" : title_in_list, "URL": url_in_list})

    if title_in_list ==before_title_list:
        break
    before_title_list=title_in_list

    for j in range(len(this_page_article_info)):
        this_article_info=this_page_article_info.iloc[j]
        this_article_date = this_article_info["날짜"]
        this_article_title = this_article_info["제목"]
        this_article_url = this_article_info["URL"]
        if do_all==True:
            if str(end_time) not in this_article_date:
                target_article_df=target_article_df.append(this_article_info,ignore_index=True)
        else:
            if this_article_title not in all_title:
                if this_article_url not in all_url:
                    target_article_df = target_article_df.append(this_article_info, ignore_index=True)
                else:
                    stop_page=True
                    break
            else:
                stop_page=True
                break
    #특정페이지에서 멈추고 싶을때 사용
    # if i==5:
    #     break
    if stop_page==False:
        i=i+1
    else:
        break
crawling_result=pd.DataFrame()

for j in range(len(target_article_df)):
    print("inews24 - it 오피니언",j+1 ,"//",len(target_article_df), "수집중...")
    target_article_date = target_article_df.iloc[j]["날짜"]
    target_article_title = target_article_df.iloc[j]["제목"]
    target_article_url = target_article_df.iloc[j]["URL"]
    driver.get(target_article_url)
    contents_area_html=driver.find_elements_by_css_selector('article > p')
    contents_all=[vl.text for vl in contents_area_html]
    contents_all="\n".join(contents_all)
    contents_split = contents_all.split('\n')
    md_contents_list = []
    for contents_by_line in contents_split:
        #picture_regex로 사진아래 범례를 모두 삭제함
        contents_by_line=re.sub(inews_regex,'',contents_by_line)
        contents_by_line = contents_by_line.strip()
        if contents_by_line !='':
            md_contents_list.append(contents_by_line)
    md_contents='|'.join(md_contents_list)
    this_target_article_df=pd.DataFrame({"journal":["inews24"],"type":"기자수첩","date":[target_article_date],"title":[target_article_title],"text":[md_contents],"url":[target_article_url]
                                         })
    crawling_result=crawling_result.append(this_target_article_df)

crawling_result = crawling_result.sort_values(by="date", ascending=True)
crawling_result = crawling_result.reset_index(drop=True)
all_data=all_data.append(crawling_result,ignore_index=True)
all_data.to_excel('./Web_Article/inews24/inews24_기자수첩.xlsx',index=False)
driver.close()

inews24 - it 오피니언 1 번째 페이지 확인중 ...
inews24 - it 오피니언 2 번째 페이지 확인중 ...
inews24 - it 오피니언 3 번째 페이지 확인중 ...
inews24 - it 오피니언 4 번째 페이지 확인중 ...
inews24 - it 오피니언 5 번째 페이지 확인중 ...
inews24 - it 오피니언 6 번째 페이지 확인중 ...
inews24 - it 오피니언 1 // 100 수집중...
inews24 - it 오피니언 2 // 100 수집중...
inews24 - it 오피니언 3 // 100 수집중...
inews24 - it 오피니언 4 // 100 수집중...
inews24 - it 오피니언 5 // 100 수집중...
inews24 - it 오피니언 6 // 100 수집중...
inews24 - it 오피니언 7 // 100 수집중...
inews24 - it 오피니언 8 // 100 수집중...
inews24 - it 오피니언 9 // 100 수집중...
inews24 - it 오피니언 10 // 100 수집중...
inews24 - it 오피니언 11 // 100 수집중...
inews24 - it 오피니언 12 // 100 수집중...
inews24 - it 오피니언 13 // 100 수집중...
inews24 - it 오피니언 14 // 100 수집중...
inews24 - it 오피니언 15 // 100 수집중...
inews24 - it 오피니언 16 // 100 수집중...
inews24 - it 오피니언 17 // 100 수집중...
inews24 - it 오피니언 18 // 100 수집중...
inews24 - it 오피니언 19 // 100 수집중...
inews24 - it 오피니언 20 // 100 수집중...
inews24 - it 오피니언 21 // 100 수집중...
inews24 - it 오피니언 22 // 100 수집중...
inews24 - it 오피니언 23 // 100 수